In [1]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sklearn.model_selection import train_test_split
from creme import metrics
import creme
from creme import naive_bayes
from creme import feature_extraction
from creme import compose
import math
import pickle
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# stopwords.words('english')
# string.punctuation
ps = PorterStemmer()
# ps.stem('worries')
import nltk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dilshad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dilshad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load the model from the file
# with open('predict-doc.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

In [4]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [5]:
def preprocesse_df(df):
    df = df.dropna()
    df.duplicated().sum()
    df = df.drop_duplicates(keep='first')
    df['transformed_content'] = df['content'].apply(transform_text)
    processed_df = df[df['content'].str.len() >= 1000]

    return processed_df

In [7]:
import os
import numpy as np

# Define the path to the train folder
train_folder_path = './db/train/'

# Get a list of all filenames in the train folder
filenames = os.listdir(train_folder_path)

# Filter out only the files (excluding directories)
filenames = [filename for filename in filenames if os.path.isfile(
    os.path.join(train_folder_path, filename))]

# Convert the list of filenames to a NumPy array
filenames_array = np.array(filenames)

# Display the filenames array
print(filenames_array)

['Business Documents.csv' 'Creative Documents.csv'
 'Educational Documents.csv' 'Financial Documents.csv'
 'Govt Documents.csv' 'Legal Documents.csv' 'Medical Documents.csv'
 'News Documents.csv' 'Scientific Documents.csv' 'Technical Documents.csv']


In [10]:
df = pd.read_csv('./db/train/'+filenames_array[0])
df

,content,category
0,Message-ID: <24216240.1075855687451.JavaMail.e...,NaN
1,Message-ID: <25140503.1075855687800.JavaMail.e...,NaN
2,Message-ID: <19034252.1075855687825.JavaMail.e...,NaN
3,Message-ID: <719350.1075855687850.JavaMail.eva...,NaN
4,Message-ID: <10523086.1075855687873.JavaMail.e...,NaN
...,...,...
1011,Message-ID: <15816310.1075855374294.JavaMail.e...,NaN
1012,Message-ID: <6521706.1075855374316.JavaMail.ev...,NaN
1013,Message-ID: <21543395.1075855374340.JavaMail.e...,NaN
1014,Message-ID: <25363451.1075855374674.JavaMail.e...,NaN


In [ ]:
# Load the model from the file
# with open('predict-doc2.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

# model = loaded_model

In [18]:
# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels = []

In [17]:
for i,file in enumerate(filenames_array):
    if i<=8:
        continue
    print(i, file, 'started')
    df = pd.read_csv('./db/train/'+file)

    if i==5:
        del df['Unnamed: 2']
        del df['Unnamed: 3']
        del df['Legal Document']

    if i==8:
        df.loc[:, 'category'] = 'Scientific Research Paper'

    # df.rename(columns={'Content': 'content',
    #           'Category': 'category'}, inplace=True)
    if file == 'Business Documents.csv':
        df.loc[:, 'category'] = 'Business Documents'

    new_df = preprocesse_df(df)

    transformed_df = new_df[['transformed_content', 'category']]
    print(transformed_df.shape)

    # message_train, message_test = train_test_split(transformed_df)

    messages_train = transformed_df.to_records(index=False)
    # messages_test = message_test.to_records(index=False)

    metric = metrics.Accuracy()
    # Training the model row by row
    for content, category in messages_train:
        model = model.fit_one(content, category)
        y_pred = model.predict_one(content)
        metric = metric.update(category, y_pred)
        true_labels.append(category)
        predicted_labels.append(y_pred)
    print(metric)

    # # test Data Accuracy
    # test_metric = metrics.Accuracy()
    # for content, category in messages_test:
    #     y_pred = model.predict_one(content)
    #     test_metric = metric.update(category, y_pred)
    # print(test_metric)

    print(i, file, 'successful ------------------------------')

8 Scientific Documents.csv started
(7, 2)
Accuracy: 71.43%
8 Scientific Documents.csv successful ------------------------------
9 Technical Documents.csv started
(10, 2)
Accuracy: 100.00%
9 Technical Documents.csv successful ------------------------------


In [ ]:
with open('predict-doc2.pkl', 'wb') as f:
    pickle.dump(model, f)